In [21]:
# import dependencies
import time
from pprint import pprint
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import calendar

In [22]:
rc =pd.read_csv('Racetrack_data.csv', index_col=0)

In [23]:
rc

,Track,Trk Length,Left_Corners,Right_Corners,track_width,length of longest straight,MotoGP_avg_speed,GP_distance,Moto2_distance,Moto3_distance
0,CAT - Circuit de Barcelona-Catalunya,4.6000,6.0,8.0,12.0,1047.0,164.4,111.0,101.8,97.2
1,JPN - Twin Ring Motegi,4.8000,6.0,8.0,15.0,762.0,162.2,115.2,105.6,96.0
2,GER - Sachsenring,3.7000,10.0,3.0,12.0,700.0,160.6,110.1,102.8,99.1
3,AME - Circuit Of The Americas,5.5000,11.0,9.0,15.0,1200.0,158.4,110.3,99.2,93.7
4,ARA - MotorLand Aragón,5.1000,10.0,7.0,15.0,968.0,167.0,116.8,106.6,96.5
5,CZE - Automotodrom Brno,5.4000,6.0,8.0,15.0,636.0,165.5,113.5,102.7,97.3
6,NED - TT Circuit Assen,4.5000,6.0,12.0,14.0,487.0,173.1,118.1,109.0,99.9
7,VAL - Circuit Ricardo Tormo,4.0000,9.0,5.0,12.0,876.0,139.8,108.1,100.1,92.1
8,RSM - Misano World Circuit Marco Simoncelli,4.2000,6.0,10.0,14.0,530.0,162.6,114.1,105.7,97.2
9,MAL - Sepang International Circuit,5.5000,5.0,10.0,16.0,920.0,164.0,110.9,99.8,94.2


In [28]:
years=['2002','2003','2004','2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017','2018']




# function to parse dates as we read in results csv files
def dateparser(dstr):
    """ Returns a datetime object for any date string in the format
        Month, dd, yyyy """
    d = dict((v,k) for k,v in enumerate(calendar.month_name))
    if type(dstr) != float:
        mon, dd, yyyy = dstr.split(',')
        date_str = '/'.join([str(d[mon]), dd, yyyy])
        return pd.to_datetime(date_str, format='%m/%d/%Y')
    else:
        pass
    return None

# function that'll help determine which string values are actually numbers
def isNumber(x):
    if pd.isnull(x):
        return False
    try:
        float(x)
        return True
    except ValueError:
        pass
    return False

def subset(string, chars):
    if pd.isnull(string):
        return None
    else:
        return string[:chars]

# function that'll convert finishing time into a proper timedelta value
def timeparser(time_series):
    
    new_time = []
    for index, item in enumerate(time_series):
        prev_case = True
        if pd.isnull(item):
            new_time.append(None)
            continue
        
        if 'Lap' in item:
            new_time.append(None)
            prev_case = False
            
        if ('+' in item) & ("'" not in item):
            t = datetime.strptime(item, "+%S.%f")
            t_delta = timedelta(hours=t.hour, minutes=t.minute,seconds=t.second, microseconds=t.microsecond)
            t_delta = t_delta + base_delta
            new_time.append(t_delta)
            prev_case = False
            
        if ('+' in item) & ("'" in item):
            t = datetime.strptime(item, "+%M'%S.%f")
            t_delta = timedelta(hours=t.hour, minutes=t.minute,seconds=t.second, microseconds=t.microsecond)
            t_delta = t_delta + base_delta
            new_time.append(t_delta)
            prev_case = False
            
        if prev_case:
            base_time = datetime.strptime(item, "%M'%S.%f")
            base_delta = timedelta(hours=base_time.hour, minutes=base_time.minute,
                                   seconds=base_time.second, microseconds=base_time.microsecond)
            new_time.append(base_delta)
            
    return new_time

In [29]:
dfs = []
rows_read = []
for yr in reversed(years):
    print(yr, end=', ')
    df = pd.read_csv('./archive/'+yr+'_data.csv', index_col=0, parse_dates=['Date'], date_parser=dateparser)
#     df['Track_Temp'] = df['Track_Temp'].map(lambda x: int(x[:2]) if isNumber(subset(x,2)) else x)
#     df['Air_Temp'] = df['Air_Temp'].map(lambda x: int(x[:2]) if isNumber(subset(x,2)) else x)
#     df['Humidity'] = df['Humidity'].map(lambda x: float(x[:2])/100 if isNumber(subset(x,2)) else x)
    df['Finish_Time'] = timeparser(df.Time)
    df['Track'] = df.TRK.map(lambda x: x+' - ') + df.Track.map(lambda x: x.split(' - ')[1])
    df = df.merge(rc, on='Track', how='left')
    rows_read.append(len(df))
    dfs.append(df)
print('Completed Time Conversion and Merge!!')

2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, Completed Time Conversion and Merge!!


In [30]:
result = pd.concat(dfs, ignore_index=True)
fn = 'MotoGP_2002_2018.csv'
result.to_csv(fn)

In [32]:
pd.read_csv('MotoGP_2002_2018.csv').head(50)

,Unnamed: 0,Year,TRK,Track,Category,Session,Date,Track_Condition,Track_Temp,Air_Temp,...,Finish_Time,Trk Length,Left_Corners,Right_Corners,track_width,length of longest straight,MotoGP_avg_speed,GP_distance,Moto2_distance,Moto3_distance
0,0,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:34.654000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
1,1,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:34.681000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
2,2,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:35.451000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
3,3,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:37.535000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
4,4,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:38.475000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
5,5,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:38.542000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
6,6,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:39.275000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
7,7,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:41.766000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
8,8,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:47.611000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8
9,9,2018,QAT,QAT - Losail International Circuit,MotoGP,RAC,2018-03-18,Dry,23.0,21.0,...,0 days 00:42:49.248000000,5.4,6.0,10.0,12.0,1068.0,166.6,118.4,107.6,96.8


In [33]:
from sklearn.preprocessing import PolynomialFeatures

interactions = PolynomialFeatures(degree=3)

kf = KFold(n_splits=5, shuffle=True, random_state=0)
pred_R2s = []
pred_RMSEs = []
i = 1

plt.figure(figsize=(25,5))
for train_i, test_i in kf.split(x, y):
    
    lm = LinearRegression()
    lm.fit(interactions.fit_transform(x.iloc[train_i]), y.iloc[train_i])
    y_pred = lm.predict(interactions.fit_transform(x.iloc[test_i]))
    
    plt.subplot(1, 5, i)
    plt.ylim(ymin = -150000, ymax=150000)
    if i == 1:
        plt.ylabel('Residual')
    plt.xlabel('Predicted Values')
    plt.scatter(y_pred, y.iloc[test_i]-y_pred, alpha=0.1);
    i += 1
    
    pred_R2s.append(lm.score(interactions.fit_transform(x.iloc[test_i]), y.iloc[test_i]))
    pred_RMSEs.append(np.sqrt(np.mean((y_pred - y.iloc[test_i])**2)))

NameError: name 'KFold' is not defined